In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
# Creating spark
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [3]:
# Reading crimes
crimes = spark.read.csv("../data/reported-crimes.csv", inferSchema=True, header=True)
crimes.limit(5)

ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
10224738,HY411648,09/05/2015 01:30:...,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY ...,RESIDENCE,false,true,924,9,12,61,08B,1165074,1875917,2015,02/10/2018 03:50:...,41.815117282,-87.669999562,"(41.815117282, -8..."
10224739,HY411615,09/04/2015 11:30:...,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,false,false,1511,15,29,25,06,1138875,1904869,2015,02/10/2018 03:50:...,41.895080471,-87.765400451,"(41.895080471, -8..."
11646166,JC213529,09/01/2018 12:01:...,082XX S INGLESIDE...,0810,THEFT,OVER $500,RESIDENCE,false,true,631,6,8,44,06,null,null,2018,04/06/2019 04:04:...,null,null,null
10224740,HY411595,09/05/2015 12:45:...,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/...,SIDEWALK,true,false,1412,14,35,21,18,1152037,1920384,2015,02/10/2018 03:50:...,41.937405765,-87.716649687,"(41.937405765, -8..."
10224741,HY411610,09/05/2015 01:00:...,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,false,true,1522,15,28,25,08A,1141706,1900086,2015,02/10/2018 03:50:...,41.881903443,-87.755121152,"(41.881903443, -8..."


In [4]:
# Question 1: Find the most frequently reported non-criminal activity

In [5]:
non_criminal_types = crimes.select("Primary Type").distinct().filter(f.col("Primary Type").contains("NON"))
non_criminal_types

Primary Type
NON-CRIMINAL (SUB...
NON - CRIMINAL
NON-CRIMINAL


In [6]:
most_frequent = (
    crimes
    .join(non_criminal_types, "Primary Type", "inner")
    .select("Description")
    .groupBy("Description")
    .count()
    .sort("count", ascending=False)
)
most_frequent

Description,count
LOST PASSPORT,85
FOID - REVOCATION,51
NOTIFICATION OF C...,6
NOTIFICATION OF S...,6
CONCEALED CARRY L...,5
FOUND PASSPORT,2
GUN OFFENDER NOTI...,1


In [7]:
# Question 2: Find the day of the week with the most reported crime

In [8]:
crime_dates = crimes.select("Date").filter("Date is not NULL")  # because there are null cells
crime_dates = crime_dates.select(f.to_date("Date", "MM/dd/yyyy hh:mm:ss a").alias("Date"))
crime_days = crime_dates.select(f.date_format("Date", "E").alias("Day"))
crime_days.limit(5)

Day
Sat
Fri
Sat
Sat
Sat


In [9]:
(
    crime_days
    .groupBy("Day")
    .count()
    .sort("count", ascending=False)
    .limit(1)
)

Day,count
Fri,586051
